# 코로나 확진자 추세에 따른 카드매출발생금액 예측
(3) 예측 모델링

## Preprocessings

In [1]:
# import required packages
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
plt.rc('font', family='AppleGothic')

In [2]:
# directory
current_path = os.getcwd()
output_DIR = os.path.join(current_path, 'output')


# data path
pinfo_path = os.path.join(output_DIR, 'pinfo_df.csv')
card_path = os.path.join(output_DIR, 'card_df.csv')


# load data
pinfo_df = pd.read_csv(pinfo_path)
card_df = pd.read_csv(card_path)

In [3]:
# check dataframe
pinfo_df.head()

,patient_id,age,male,city_nm,infection_case,confirmed_date,state,confirmed
0,1000000001,50s,1.0,강서구,overseas inflow,2020-01-23,released,1
1,1000000002,30s,1.0,중랑구,overseas inflow,2020-01-30,released,1
2,1000000003,50s,1.0,종로구,contact with patient,2020-01-30,released,1
3,1000000004,20s,1.0,마포구,overseas inflow,2020-01-30,released,1
4,1000000005,20s,0.0,성북구,contact with patient,2020-01-31,released,1


In [4]:
# check dataframe
card_df.head()

,date,month,city_code,city_nm,category,induty_nm,sales_count,sales
0,2020-01-04,1,11740,강동구,병원,약국,463,5843230
1,2020-01-04,1,11740,강동구,병원,치과의원,33,7835550
2,2020-01-04,1,11740,강동구,병원,한의원,53,4589800
3,2020-01-04,1,11740,강동구,병원,의원,339,9267240
4,2020-01-04,1,11740,강동구,차량,자동차정비,19,4441000


In [5]:
# group patient infromation dataframe
groups = [pinfo_df['confirmed_date'], pinfo_df['city_nm']]
confirmed = pinfo_df['confirmed'].groupby(groups).sum()
confirmed = pd.DataFrame(confirmed)
confirmed = confirmed.reset_index()
confirmed = confirmed.rename(columns={'confirmed_date':'date'})

In [6]:
# group card sales dataframe
groups = [card_df['date'], card_df['city_nm']]
sales = card_df['sales'].groupby(groups).sum()
sales = pd.DataFrame(sales)

sales_count = card_df['sales_count'].groupby(groups).sum()
sales_count = pd.DataFrame(sales_count)

card = sales.join(sales_count)
card = card.reset_index()

In [7]:
# merge dataframe
merge_df = pd.merge(card, confirmed, on=['date', 'city_nm'])

`pd.merge()` : `on` 공통 열을 기준으로 데이터프레임 병합

In [8]:
# check length of dataframe
print('confirmed 시작일:', confirmed['date'].min())
print('confirmed 종료일:', confirmed['date'].max())
print('confirmed 길이:', len(confirmed))
print()
print('card 시작일:', card['date'].min())
print('card 종료일:', card['date'].max())
print('card 길이:', len(card))
print()
print('merge 시작일:', merge_df['date'].min())
print('merge 종료일:', merge_df['date'].max())
print('merge 길이:', len(merge_df))

confirmed 시작일: 2020-01-23
confirmed 종료일: 2020-06-29
confirmed 길이: 759

card 시작일: 2020-01-04
card 종료일: 2020-06-14
card 길이: 3875

merge 시작일: 2020-01-23
merge 종료일: 2020-06-14
merge 길이: 652


In [9]:
# set index
merge_df = merge_df.set_index('date')

In [10]:
# check dataframe
merge_df.head()
# merge_df.tail()

,city_nm,sales,sales_count,confirmed
date,,,,
2020-01-23,강서구,12938184557,196724,1
2020-01-30,마포구,18707460485,511368,1
2020-01-30,종로구,13949766077,189029,1
2020-01-30,중랑구,1723337781,63246,1
2020-01-31,성북구,1844562764,74844,1


In [11]:
# check statistics
merge_df.describe()

,sales,sales_count,confirmed
count,6.520000e+02,6.520000e+02,652.000000
mean,6.555286e+09,1.968122e+05,1.613497
std,1.019199e+10,3.071721e+05,1.244713
min,5.813526e+08,2.765500e+04,1.000000
25%,1.948648e+09,8.185075e+04,1.000000
50%,2.825033e+09,1.163620e+05,1.000000
75%,6.732432e+09,1.852855e+05,2.000000
max,1.273518e+11,5.001972e+06,12.000000


In [12]:
# normalization
# it is inappropriate to create dummy variables, since there is too many cities
nr = Normalizer(copy=False)

X = merge_df.drop(columns=['sales', 'city_nm'])
X = nr.fit_transform(X)
y = merge_df['sales']

# data partition
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.3, random_state=123)

## Baseline

In [13]:
def evaluate(model):
    # returns R2 score and RMSE of validation set
    from sklearn.metrics import r2_score, mean_squared_error
    from math import sqrt
    
    model.fit(train_X, train_y)
    prediction = model.predict(valid_X)
    
    R2 = np.mean(r2_score(valid_y, prediction))
    RMSE = sqrt(mean_squared_error(valid_y, prediction))
    
    print('R2 Score = %.2f'%R2)
    print('RMSE = %.2f'%RMSE)
    return r2_score, RMSE

In [14]:
# linear regression
# train
lr = LinearRegression()
lr.fit(train_X, train_y)

# evaluate
lr_pred = lr.predict(valid_X)
lr_score = evaluate(lr)

R2 Score = -18.83
RMSE = 38180212266.43


In [15]:
# support vector machine
# train
svc = SVC()
svc.fit(train_X, train_y)

# evaluate
svc_pred = svc.predict(valid_X)
svc_score = evaluate(svc)

R2 Score = 0.99
RMSE = 703118066.95


In [16]:
# regression tree
# train
rt = DecisionTreeRegressor()
rt.fit(train_X, train_y)

# evaluate
rt_pred = rt.predict(valid_X)
rt_score = evaluate(rt)

R2 Score = 0.46
RMSE = 6320705858.65


In [17]:
# random forest
# train
rf = RandomForestRegressor()
rf.fit(train_X, train_y)

# evaluate
rf_pred = rf.predict(valid_X)
rf_score = evaluate(rf)

R2 Score = 0.50
RMSE = 6091902021.94


In [19]:
# save dataframe
merge_df.to_csv(output_DIR+'/merge_df.csv', index=False, encoding='cp949')

`encoding='cp949'`
- python에서는 `utf-8`을 사용하지만 microsoft fat 파일시스템 계열은 'cp949' 사용
- 즉, 엑셀로 열어봤을 때 한글을 깨지지 않게 하기 위함